In [1]:
import matplotlib.pyplot as plt
from tqdm import tqdm
import numpy as np
import pandas as pd
import os
import cv2
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential,Model, load_model
from tensorflow.keras.layers import (Input, Dense, Dropout, Activation, 
                                     Flatten, BatchNormalization, ZeroPadding2D,
                                     MaxPooling2D,Softmax, Convolution2D,)
from tensorflow.keras.applications.imagenet_utils import preprocess_input

In [2]:
# check GPU
tf.test.is_gpu_available()

False

In [ ]:
# 移除.ipynb_checkpoints資料夾, 避免讀取圖片時發生錯誤
import shutil

nowpath = os.getcwd()
try:   
    shutil.rmtree(nowpath +'/face_cnn_train/akane/.ipynb_checkpoints')
except:
    pass
try:   
    shutil.rmtree(nowpath +'/face_cnn_train/neru/.ipynb_checkpoints')
except:
    pass
try:   
    shutil.rmtree(nowpath +'/face_cnn_test/.ipynb_checkpoints')
except:
    pass
try:   
    shutil.rmtree(nowpath +'/face_cnn_train/rika/.ipynb_checkpoints')
except:
    pass
try:   
    shutil.rmtree(nowpath +'/face_cnn_train/risa/.ipynb_checkpoints')
except:
    pass
try:   
    shutil.rmtree(nowpath +'/face_cnn_train/yui/.ipynb_checkpoints')
except:
    pass

In [ ]:
data_path = "./face_cnn_train"

x_data_list = []
y_data_list = []
for roots, dirs, files in os.walk(data_path):
    if dirs == []:
        for each in files:
            if each.find('check') == -1:
                x_data_list.append(os.path.join(roots, each))
                y_data_list.append(roots.split("/")[-1])

In [ ]:
data_path = "./face_cnn_test"
names=[]
test_list = []
for roots, dirs, files in os.walk(data_path):
        for each in files:
            if each.find('check') == -1:
                names.append(each.split(".")[0])
                test_list.append(os.path.join(roots, each))

In [ ]:
image_size = 224

In [ ]:
def load_img(data_list):
    data_img = []
    for each in tqdm(data_list):
        img = cv2.imread(each, 1)
        img = cv2.resize(img, (image_size, image_size))
        data_img.append(img)

    return np.array(data_img)  #.astype('float32')

In [ ]:
x_data = load_img(x_data_list)

In [ ]:
y_data_list = pd.DataFrame(y_data_list, columns=['label'])
uniques = y_data_list['label'].value_counts().index

In [ ]:
uniques

In [ ]:
class_map = pd.read_csv("./classmap.csv",header=None, index_col=0)
class_map = class_map.to_dict()[1]

y_data = y_data_list['label'].map(class_map).values.copy()

In [ ]:
# preprcoess
x_data = preprocess_input(x_data)

In [ ]:
# one-hot encoding
y_data = keras.utils.to_categorical(y_data)

In [ ]:
# split training/validation set
from sklearn.model_selection import train_test_split

x_train, x_valid, y_train, y_valid = train_test_split(x_data, y_data,
                                                                                                       test_size=0.1,
                                                                                                       random_state=666,
                                                                                                       stratify=y_data)

In [ ]:
def call_list(models, model_name):
    model_dir = './Model/{}-logs'.format(model_name)
    if not os.path.exists(model_dir):
        os.makedirs(model_dir)

    logfiles = model_dir + '/{}-{}'.format('basic_model',
                                           models.__class__.__name__)
    model_cbk = keras.callbacks.TensorBoard(log_dir=logfiles,
                                            histogram_freq=1)

    modelfiles = model_dir + '/{}-best-model.h5'.format('basic_model')
    model_mckp = keras.callbacks.ModelCheckpoint(modelfiles,
                                                 monitor='val_accuracy',
                                                 save_best_only=True)

    return [model_cbk, model_mckp]

In [ ]:
# Define VGG_FACE_MODEL architecture
model = Sequential()
model.add(ZeroPadding2D((1,1),input_shape=(224,224, 3)))
model.add(Convolution2D(64, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))
model.add(ZeroPadding2D((1,1)))	
model.add(Convolution2D(128, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))
model.add(Convolution2D(4096, (7, 7), activation='relu'))
model.add(Dropout(0.5))
model.add(Convolution2D(4096, (1, 1), activation='relu'))
model.add(Dropout(0.5))
model.add(Convolution2D(2622, (1, 1)))
model.add(Flatten())
model.add(Activation('softmax'))

# Load VGG Face model weights
model.load_weights('./vgg_face_weights.h5')

In [ ]:
# Remove last Softmax layer and get model upto last flatten layer
vgg_face=Model(inputs=model.layers[0].input,outputs=model.layers[-2].output) 

In [ ]:
classifier_model=Sequential()
classifier_model.add(vgg_face)
classifier_model.add(Dense(units=64,kernel_initializer='he_uniform'))
classifier_model.add(BatchNormalization())
classifier_model.add(Activation('relu'))
classifier_model.add(Dropout(0.2))
classifier_model.add(Dense(units=32,kernel_initializer='he_uniform'))
classifier_model.add(BatchNormalization())
classifier_model.add(Activation('relu'))
classifier_model.add(Dropout(0.2))
classifier_model.add(Dense(units=16,kernel_initializer='he_uniform'))
classifier_model.add(BatchNormalization())
classifier_model.add(Activation('relu'))
classifier_model.add(Dropout(0.2))
classifier_model.add(Dense(units=5,kernel_initializer='he_uniform'))
classifier_model.add(Activation('softmax'))

In [ ]:
classifier_model.compile(loss=tf.keras.losses.categorical_crossentropy,
                                                    optimizer=keras.optimizers.Adam(lr=5e-5, decay = 1e-8),
                                                    metrics=['accuracy'])

In [ ]:
print("凍結前可調整權重的層數：", len(classifier_model.trainable_weights))

vgg_face.trainable = False

print("凍結後可調整權重的層數：", len(classifier_model.trainable_weights))

In [ ]:
call_backs_face = call_list(classifier_model, "vgg_face")

In [ ]:
batch_size = 20
epo = 30
num_step = x_train.shape[0] // batch_size + 1

In [ ]:
# from tensorflow.keras.preprocessing.image import ImageDataGenerator

# datagen = ImageDataGenerator(
#                              width_shift_range=0.1,
#                              height_shift_range=0.1,
#                              horizontal_flip=True,
#                              vertical_flip=False,
#                              fill_mode='nearest'
#                              )

In [ ]:
history_face = classifier_model.fit(x_train, y_train,
                                                                      batch_size=batch_size,
                                                                      epochs=epo,
                                                                      validation_data=(x_valid, y_valid),
                                                                      callbacks=call_backs_face)

In [ ]:
model_face = load_model('./Model/vgg_face-logs/basic_model-best-model.h5')

In [ ]:
test_data = load_img(test_list)
test_data = preprocess_input(test_data)

In [ ]:
y_pred = model_face.predict(test_data)
predictions = np.argmax(y_pred, 1)

In [ ]:
test_submission = pd.DataFrame({'Id':names, 'class': predictions})
test_submission.to_csv('./try.csv', index = False)

#### Final result  
**Public : 0.96183**  
**Private: 0.88599**